In [11]:
import statistics
import random
import pyforest
from lazypredict.Supervised import LazyClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score

<h2> Data Preparation <h2>

In [2]:
# load Data

df = pd.read_csv('weatherAUS.csv')
df.head()

<IPython.core.display.Javascript object>

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.40,22.90,0.60,nan,nan,W,44.00,W,...,71.00,22.00,1007.70,1007.10,8.00,nan,16.90,21.80,No,No
1,2008-12-02,Albury,7.40,25.10,0.00,nan,nan,WNW,44.00,NNW,...,44.00,25.00,1010.60,1007.80,nan,nan,17.20,24.30,No,No
2,2008-12-03,Albury,12.90,25.70,0.00,nan,nan,WSW,46.00,W,...,38.00,30.00,1007.60,1008.70,nan,2.00,21.00,23.20,No,No
3,2008-12-04,Albury,9.20,28.00,0.00,nan,nan,NE,24.00,SE,...,45.00,16.00,1017.60,1012.80,nan,nan,18.10,26.50,No,No
4,2008-12-05,Albury,17.50,32.30,1.00,nan,nan,W,41.00,ENE,...,82.00,33.00,1010.80,1006.00,7.00,8.00,17.80,29.70,No,No


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [4]:
df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [5]:
# Drop Null values

df = df.dropna(axis='index')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,2009-01-01,Cobar,17.90,35.20,0.00,12.00,12.30,SSW,48.00,ENE,...,20.00,13.00,1006.30,1004.40,2.00,5.00,26.60,33.40,No,No
6050,2009-01-02,Cobar,18.40,28.90,0.00,14.80,13.00,S,37.00,SSE,...,30.00,8.00,1012.90,1012.10,1.00,1.00,20.30,27.00,No,No
6052,2009-01-04,Cobar,19.40,37.60,0.00,10.80,10.60,NNE,46.00,NNE,...,42.00,22.00,1012.30,1009.20,1.00,6.00,28.70,34.90,No,No
6053,2009-01-05,Cobar,21.90,38.40,0.00,11.40,12.20,WNW,31.00,WNW,...,37.00,22.00,1012.70,1009.10,1.00,5.00,29.10,35.60,No,No
6054,2009-01-06,Cobar,24.20,41.00,0.00,11.20,8.40,WNW,35.00,NW,...,19.00,15.00,1010.70,1007.40,1.00,6.00,33.60,37.60,No,No


In [6]:
df['Month'] = df['Date'].apply(lambda x: int(x.split('-')[1]))
df = df.drop('Date', axis='columns')
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Month
6049,Cobar,17.90,35.20,0.00,12.00,12.30,SSW,48.00,ENE,SW,...,13.00,1006.30,1004.40,2.00,5.00,26.60,33.40,No,No,1
6050,Cobar,18.40,28.90,0.00,14.80,13.00,S,37.00,SSE,SSE,...,8.00,1012.90,1012.10,1.00,1.00,20.30,27.00,No,No,1
6052,Cobar,19.40,37.60,0.00,10.80,10.60,NNE,46.00,NNE,NNW,...,22.00,1012.30,1009.20,1.00,6.00,28.70,34.90,No,No,1
6053,Cobar,21.90,38.40,0.00,11.40,12.20,WNW,31.00,WNW,WSW,...,22.00,1012.70,1009.10,1.00,5.00,29.10,35.60,No,No,1
6054,Cobar,24.20,41.00,0.00,11.20,8.40,WNW,35.00,NW,WNW,...,15.00,1010.70,1007.40,1.00,6.00,33.60,37.60,No,No,1


In [7]:
# Encode Target

target_encode = LabelEncoder()
y = target_encode.fit_transform(df['RainTomorrow'])

In [8]:
# Encode

X = df.drop(['RainTomorrow'], axis='columns')
X = pd.get_dummies(X, drop_first=True)
X.info()

<IPython.core.display.Javascript object>

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56420 entries, 6049 to 142302
Data columns (total 88 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   MinTemp                    56420 non-null  float64
 1   MaxTemp                    56420 non-null  float64
 2   Rainfall                   56420 non-null  float64
 3   Evaporation                56420 non-null  float64
 4   Sunshine                   56420 non-null  float64
 5   WindGustSpeed              56420 non-null  float64
 6   WindSpeed9am               56420 non-null  float64
 7   WindSpeed3pm               56420 non-null  float64
 8   Humidity9am                56420 non-null  float64
 9   Humidity3pm                56420 non-null  float64
 10  Pressure9am                56420 non-null  float64
 11  Pressure3pm                56420 non-null  float64
 12  Cloud9am                   56420 non-null  float64
 13  Cloud3pm                   56420 non-null 

In [9]:
# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<IPython.core.display.Javascript object>

<h2> Model Selection <h2>

In [10]:
# Model Selection

models_dict = {}
random_state_list =[]

for i in range(5):
    # Train Validation Split
    rand_state = random.randint(0,1000)
    random_state_list.append(rand_state)
    X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=rand_state)
    
    # Traning model
    clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
    models_dict["model{}".format(i+1)], predicitons = clf.fit(X_train_val, X_val, y_train_val, y_val)

<IPython.core.display.Javascript object>

100%|██████████| 29/29 [06:46<00:00, 14.01s/it]


<IPython.core.display.Javascript object>

100%|██████████| 29/29 [07:10<00:00, 14.83s/it]


<IPython.core.display.Javascript object>

100%|██████████| 29/29 [06:57<00:00, 14.38s/it]


<IPython.core.display.Javascript object>

100%|██████████| 29/29 [06:51<00:00, 14.19s/it]


<IPython.core.display.Javascript object>

100%|██████████| 29/29 [06:55<00:00, 14.31s/it]


In [12]:
models_dict.keys()

dict_keys(['model1', 'model2', 'model3', 'model4', 'model5'])

In [17]:
# Finding Best Model

average_model = (models_dict['model1'] + models_dict['model2'] + models_dict['model3'] + models_dict['model4']  + models_dict['model5'])/5

average_model.sort_values(['Accuracy', 'Balanced Accuracy'], axis=0, ascending=False)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.86,0.75,0.75,0.86,117.74
LGBMClassifier,0.86,0.76,0.76,0.86,0.91
XGBClassifier,0.86,0.76,0.76,0.86,3.90
ExtraTreesClassifier,0.86,0.73,0.73,0.85,6.40
RandomForestClassifier,0.86,0.74,0.74,0.85,6.09
LogisticRegression,0.86,0.75,0.75,0.85,0.49
LinearSVC,0.86,0.74,0.74,0.85,9.03
CalibratedClassifierCV,0.86,0.74,0.74,0.85,33.87
RidgeClassifier,0.86,0.73,0.73,0.84,0.19


SVC, LGBMClassifier and XGBClassifier performed similarly. However, LGBMClassifier took the least time. Thus, LGBMClassifier is chosen as the final model.

<h2> Hyperparameter optimization <h2>

In [47]:
lightgmb = lgb.LGBMClassifier(num_threads = 8, 
                              boosting_type = 'gbdt',
                              learning_rate = 0.1,
                              max_depth = -1,
                              objective = 'binary',
                              min_split_gain = 0,
                              class_weight = None)

params = {
    'num_leaves':[25,30,35,40],
    'n_estimators':[300,400,500,600]
}

grid_lightgmb = sklearn.model_selection.GridSearchCV(lightgmb, params, cv=10)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
grid_lightgmb.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=LGBMClassifier(min_split_gain=0, num_threads=8,
                                      objective='binary'),
             param_grid={'n_estimators': [300, 400, 500, 600],
                         'num_leaves': [25, 30, 35, 40]})

In [51]:
grid_lightgmb.best_params_

{'n_estimators': 500, 'num_leaves': 30}

In [52]:
# Evaluating Model

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

y_predict = grid_lightgmb.predict(X_test)

print('Accuracy score: ', format(accuracy_score(y_test, y_predict)))
print('Precision score: ', format(precision_score(y_test, y_predict)))
print('Recall score: ', format(recall_score(y_test, y_predict)))
print('ROC AUC score: ', format(roc_auc_score(y_test, y_predict)))
print('F1 score: ', format(f1_score(y_test, y_predict)))

Accuracy score:  0.8690180786954981
Precision score:  0.7609391675560299
Recall score:  0.5806188925081434
ROC AUC score:  0.7649356356514437
F1 score:  0.6586605080831409
